# OpenClassrooms - Ingenieur IA
# Projet 6 - Avis Restau
# Améliorez le produit IA de votre start-up

## Objectif du projet : 
- **Détecter les sujets d’insatisfaction présents dans les commentaires postés sur la plateforme**
- **Labelliser automatiquement les photos postées sur la plateforme**
- **Collecter de nouvelles données via l’API Yelp**

## Trois parties :
- **Partie A : analyser les commentaires pour détecter les différents sujets d’insatisfaction**
- **Partie B : analyser les photos pour déterminer les catégories des photos**
- **Partie C : collecter un échantillon de données via l’API Yelp**

## Plan :
- **Partie C : collecter un échantillon de données via l’API Yelp**
    - Paramètres
    - Construction du client
    - Collecte des critiques laissées par les clients
    - Collecte des photos
    - Conclusion

In [1]:
import requests
import json
import pandas as pd
import os

In [2]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Partie C : collecter un échantillon de données via l’API Yelp

### Remarques préliminaires :
- Afin de pouvoir utiliser l'API Yelp nous avons créé un compte
- Cela nous a permis de générer une clé d'authentification sur l'API
- Pour des raisons de sécurité, cette clé d'authentification est enregistrée dans les variables d'environnement afin qu'elle n'apparaisse pas en clair dans le code
- Nous avons utilisé l'API GraphQL qui est la plus avancée : elle permet de faire des requêtes sur mesure uniquement sur les données dont nous avons besoin

## Paramètres

In [3]:
api_key = os.environ['YELP_API_KEY']

header = {'Authorization': 'bearer {}'.format(api_key),
          'Content-Type':"application/json"}

nb_restaurants = 200

## Construction du client

In [4]:
transport = RequestsHTTPTransport(url='https://api.yelp.com/v3/graphql', headers=header, use_json=True)
client = Client(transport=transport, fetch_schema_from_transport=True)

## Collecte des critiques laissées par les clients

### Requête

#### Remarques : 
- L'API renvoie un maximum de 50 données par requête
- Etant donné que nous souhaitons obtenir les données pour **200 restaurants** nous utilisons une boucle pour faire plusieurs requêtes successives
- Nous récupérons les informations suivantes :
    - identifiant du restaurant (id_restaurant)
    - note de la critique (rating)
    - texte de la critique (text)
- L'API renvoie un maximum de 3 critiques par restaurant
- L'API ne renvoie pas uniquement des restaurants car la base de données Yelp contient des informations sur d'autres types d'établissements (cliniques...)
> On va donc appliquer un filtre à la requête pour ne retenir que les restaurants

In [5]:
# Requêtes successives
data_reviews = []
for offset in range(0, nb_restaurants, 50):
    query = gql('''
    {
      search(term: "restaurants",
              location: "san francisco", 
              limit:50,
              offset:'''+str(offset)+''') {
        business {
          id
          reviews {
            text
            rating
          }
        }
      }
    }
    ''')
    
    # Exécution de la requête
    response = client.execute(query)
    
    # Ajout de la réponse au dataset 
    data_reviews+=response['search']['business']

### Construction d'un Dataframe avec les résultats

In [6]:
id_restaurant = []
reviews_ratings = []
reviews_texts = []

for i in range (len(data_reviews)):
    for j in range (len(data_reviews[i]['reviews'])):
        id_restaurant.append(data_reviews[i]['id'])
        reviews_ratings.append(data_reviews[i]['reviews'][j]['rating'])
        reviews_texts.append(data_reviews[i]['reviews'][j]['text'])
    
df_data_reviews = pd.DataFrame(list(zip(id_restaurant, reviews_ratings, reviews_texts)), columns=['id_restaurant','rating', 'text'])  

### Analyse du Dataframe contenant les résultats

In [7]:
df_data_reviews

,id_restaurant,rating,text
0,reXWH9Wo0ZTOuQsTMNOSxg,5,Our party of six had a great evening. We had r...
1,reXWH9Wo0ZTOuQsTMNOSxg,3,Went to Fable today for Brunch. I ordered the ...
2,reXWH9Wo0ZTOuQsTMNOSxg,5,"To start, Fable has so many seating options! O..."
3,-DrR38H1Abk0wCyu9XOLug,1,When we scared by the little rat family runnin...
4,-DrR38H1Abk0wCyu9XOLug,5,"I've had a lot of good brunches in SF, but thi..."
...,...,...,...
593,RxIFKVvc8iVBPIVpe55Byg,4,Quickly looked up restaurants we could walk to...
594,RxIFKVvc8iVBPIVpe55Byg,3,Every dish we order had something wrong with i...
595,vth2jtXfVUCRJeIP680CyA,5,Always a joy coming here. Every dish I've ever...
596,vth2jtXfVUCRJeIP680CyA,1,They don't substitute anything. And the waiter...


#### Nombre de critiques

In [8]:
print(f"Le Dataframe contient {df_data_reviews.shape[0]} critiques")

Le Dataframe contient 598 critiques


#### Nombre de restaurants

In [9]:
nb_rest = df_data_reviews['id_restaurant'].nunique()
nb_rest

200

In [10]:
print(f"Le Dataframe contient des critiques sur {nb_rest} restaurants")

Le Dataframe contient des critiques sur 200 restaurants


### Sauvegarde des informations dans un fichier CSV

In [11]:
df_data_reviews.to_csv('data_reviews_api.csv')

### Téléchargement du fichier pour vérification

In [12]:
df_data_reviews_download = pd.read_csv('data_reviews_api.csv', index_col=0)

In [13]:
df_data_reviews_download

,id_restaurant,rating,text
0,reXWH9Wo0ZTOuQsTMNOSxg,5,Our party of six had a great evening. We had r...
1,reXWH9Wo0ZTOuQsTMNOSxg,3,Went to Fable today for Brunch. I ordered the ...
2,reXWH9Wo0ZTOuQsTMNOSxg,5,"To start, Fable has so many seating options! O..."
3,-DrR38H1Abk0wCyu9XOLug,1,When we scared by the little rat family runnin...
4,-DrR38H1Abk0wCyu9XOLug,5,"I've had a lot of good brunches in SF, but thi..."
...,...,...,...
593,RxIFKVvc8iVBPIVpe55Byg,4,Quickly looked up restaurants we could walk to...
594,RxIFKVvc8iVBPIVpe55Byg,3,Every dish we order had something wrong with i...
595,vth2jtXfVUCRJeIP680CyA,5,Always a joy coming here. Every dish I've ever...
596,vth2jtXfVUCRJeIP680CyA,1,They don't substitute anything. And the waiter...


## Collecte des photos

### Requête

#### Remarques : 
- L'API renvoie un maximum de 50 données par requête
- Etant donné que nous souhaitons obtenir les données pour **200 restaurants** nous utilisons une boucle pour faire plusieurs requêtes successive
- Nous récupérons les informations suivantes :
    - identifiant du restaurant (id_restaurant)
    - url de la photo (photo_url)

In [14]:
# Requêtes successives
data_photos = []
for offset in range(0, nb_restaurants, 50):
    query = gql('''
    {
      search(term: "restaurants",
              location:"san francisco", 
              limit:50,
              offset:'''+str(offset)+''') {
        business {
            id
            photos
          }
        }
    }
    ''')
    
    # Exécution de la requête
    response = client.execute(query)
    
    # Ajout de la réponse au dataset 
    data_photos+=response['search']['business']

### Construction d'un Dataframe avec les résultats

In [15]:
id_restaurant = []
url_photos = []

for i in range (len(data_photos)):
    id_restaurant.append(data_photos[i]['id'])
    url_photos.append(data_photos[i]['photos'])
    
df_data_photos = pd.DataFrame(list(zip(id_restaurant, url_photos)), columns=['id_restaurant','url_photos'])  

### Analyse du Dataframe contenant les résultats

In [16]:
df_data_photos

,id_restaurant,url_photos
0,reXWH9Wo0ZTOuQsTMNOSxg,[https://s3-media4.fl.yelpcdn.com/bphoto/hE4xB...
1,-DrR38H1Abk0wCyu9XOLug,[https://s3-media3.fl.yelpcdn.com/bphoto/UwlH3...
2,8kck3-K4zYKTJbJko0JlXQ,[https://s3-media4.fl.yelpcdn.com/bphoto/kUlEa...
3,4KfQnlcSu4bbTqnvGdGptw,[https://s3-media2.fl.yelpcdn.com/bphoto/HKxMJ...
4,HHtpR0RslupSQ99GIIwW5A,[https://s3-media4.fl.yelpcdn.com/bphoto/ouK2V...
...,...,...
195,6dpogM9AfMtNcXswIDJ1sg,[https://s3-media4.fl.yelpcdn.com/bphoto/LL6Lx...
196,4vqUi0Zrhj6D7nAaMZz-OQ,[https://s3-media2.fl.yelpcdn.com/bphoto/onZg_...
197,Gh3cLhqw1BqpFSN5BaemBg,[https://s3-media1.fl.yelpcdn.com/bphoto/bAfER...
198,RxIFKVvc8iVBPIVpe55Byg,[https://s3-media3.fl.yelpcdn.com/bphoto/2vFXN...


#### Nombre de photos

In [17]:
print(f"Le Dataframe contient {df_data_photos.shape[0]} photos")

Le Dataframe contient 200 photos


#### Nombre de restaurants

In [18]:
nb_rest = df_data_photos['id_restaurant'].nunique()
nb_rest

200

In [19]:
print(f"Le Dataframe contient des photos de {nb_rest} restaurants")

Le Dataframe contient des photos de 200 restaurants


### Sauvegarde des informations dans un fichier CSV

In [20]:
df_data_photos.to_csv('data_photos_api.csv')

### Téléchargement du fichier pour vérification

In [21]:
df_data_photos_download = pd.read_csv('data_photos_api.csv', index_col=0)

In [22]:
df_data_photos_download

,id_restaurant,url_photos
0,reXWH9Wo0ZTOuQsTMNOSxg,['https://s3-media4.fl.yelpcdn.com/bphoto/hE4x...
1,-DrR38H1Abk0wCyu9XOLug,['https://s3-media3.fl.yelpcdn.com/bphoto/UwlH...
2,8kck3-K4zYKTJbJko0JlXQ,['https://s3-media4.fl.yelpcdn.com/bphoto/kUlE...
3,4KfQnlcSu4bbTqnvGdGptw,['https://s3-media2.fl.yelpcdn.com/bphoto/HKxM...
4,HHtpR0RslupSQ99GIIwW5A,['https://s3-media4.fl.yelpcdn.com/bphoto/ouK2...
...,...,...
195,6dpogM9AfMtNcXswIDJ1sg,['https://s3-media4.fl.yelpcdn.com/bphoto/LL6L...
196,4vqUi0Zrhj6D7nAaMZz-OQ,['https://s3-media2.fl.yelpcdn.com/bphoto/onZg...
197,Gh3cLhqw1BqpFSN5BaemBg,['https://s3-media1.fl.yelpcdn.com/bphoto/bAfE...
198,RxIFKVvc8iVBPIVpe55Byg,['https://s3-media3.fl.yelpcdn.com/bphoto/2vFX...


## Conclusion :
- L'API GraphQL de Yelp nous a permis de récupérer :
    - les identifiants restaurants, textes et notes des critiques laissées par les clients
    - les urls des photos
- pour 200 restaurants
> **Cette étude montre que l'objectif fixé : collecter de nouvelles données via l’API Yelp, est atteignable**